In [31]:
#Import libraries
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split as tts, GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import plot_importance
from matplotlib import pyplot

In [32]:
#Load and view dataset
df = pd.read_csv('train (le).csv')
df.head()

,ID,Agency,Agency Type,Distribution Channel,Product Name,Duration,Destination,Net Sales,Commision (in value),Age,Claim
0,2010,7,1,1,10,61,68,12.0,0.00,41,0
1,4245,7,1,1,10,4,53,17.0,0.00,35,0
2,9251,6,1,1,16,26,84,19.8,11.88,47,0
3,4754,7,1,1,1,15,33,27.0,0.00,48,0
4,8840,7,1,1,1,15,53,37.0,0.00,36,0


In [33]:
#Drop ID column
df = df.drop(['ID'], 1)

In [34]:
#Split data into features and target
X = df.drop(['Claim'], 1)
y = df['Claim']

In [35]:
#Split data into training and test data
X_train, X_test, y_train, y_test = tts(X, y, test_size = 0.25, random_state = 42)

In [36]:
#Initialize gradient boosting model
gbc = GradientBoostingClassifier(n_estimators = 1000, random_state = 42, max_depth = 12, min_samples_split = 0.005)

In [37]:
#Fit model on training data
gbc.fit(X_train, y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=12,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=0.005,
                           min_weight_fraction_leaf=0.0, n_estimators=1000,
                           n_iter_no_change=None, presort='auto',
                           random_state=42, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [38]:
#Make prediction using X_test
y_pred = gbc.predict(X_test)

In [39]:
#Print classification report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.97      0.98      0.97     10902
           1       0.89      0.84      0.86      2175

    accuracy                           0.96     13077
   macro avg       0.93      0.91      0.92     13077
weighted avg       0.96      0.96      0.96     13077



In [40]:
df2 = pd.read_csv('test.csv')
validation = df2.copy()
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
#Label encode categorical features
for column in list(validation):
    if validation[column].dtypes == 'object':
        validation[column] = le.fit_transform(validation[column])

validation.head()

,ID,Agency,Agency Type,Distribution Channel,Product Name,Duration,Destination,Net Sales,Commision (in value),Age
0,17631,7,1,1,10,192,33,18.0,0.00,36
1,15064,7,1,1,0,2,75,20.0,0.00,36
2,14139,2,0,1,9,13,75,13.5,3.38,24
3,19754,7,1,1,1,133,82,41.0,0.00,36
4,16439,2,0,1,17,2,75,30.0,7.50,32


In [41]:
#Fit model on entire data
gbc.fit(X, y)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=12,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=0.005,
                           min_weight_fraction_leaf=0.0, n_estimators=1000,
                           n_iter_no_change=None, presort='auto',
                           random_state=42, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [42]:
#Drop ID column from validation set and unimportant features
validation_new = validation.drop(['ID'], 1)

In [43]:
#Use model to predict on validation set
validation_pred = gbc.predict(validation_new)

In [44]:
#Combine ID and validation Pred
ID = pd.DataFrame(validation['ID'], columns =['ID'])
claim = pd.DataFrame(validation_pred, columns = ['Claim'])
submission = ID.join(claim)

In [45]:
#Save submission
submission.to_csv('Final_Submission.csv', index = False)